In [1]:
#Just the model portion in this notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
#Cross validation - will randomly sample from training data
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
#from sklearn #import all the different types of models

from data_preprocessing import train_val_test

In [4]:
import os
cwd = os.getcwd()

In [10]:
model_data = pd.read_csv(cwd + '/data/model_data.csv')

In [8]:
model_data.head()

,Season,Gender,Age,Height,Entered,Finalist,Loyalty,Ireland,Islands,Midlands,NFR,North,Scotland,South,Wales
0,2,Female,26,66,7,No,1.0,False,False,True,False,False,False,False,False
1,1,Male,24,71,32,No,1.0,False,False,False,False,True,False,False,False
2,1,Female,22,66,32,No,1.0,False,False,False,False,False,False,True,False
3,2,Male,25,74,30,No,1.0,False,False,False,False,False,False,True,False
4,7,Female,22,68,1,No,1.0,False,False,False,False,False,True,False,False


### More Preprocessing to do
Need to change finalist to binary rather than yes/no - do this in preprocessing.

In [11]:
x_train, y_train, x_validate, y_validate, x_test, y_test = train_val_test(model_data, 'Finalist')

Train Split:  (224, 14) | Validate Split:  (36, 14) | Test Split:  (35, 14)


Seasons typically start with 11 islanders on episode 1. 5 initial couples and one bombshell. 
People are periodically added and they are usually 7  to 9 couples with 1 - 3 additional singles. Therefore there are roughly 15 - 21 contestants on at any given time. I am looking for the top 8 (4  from each gender)
I am predicting the 8 most probable islanders to make finals. I can either take their score as is, or then match go a step further and evaluate current couples likelihood.

Model types i want to try

logistic - https://medium.com/@ashishmishra2k19/logistic-regression-the-baseline-classification-model-explained-89915f5b13b2
- has some drawbacks for my specific case - it assumes a linear relationship between features, is sensitive to outliers, and  cant automatically learn features  like tree based models
- no need to scale features tho
- output is probability scores which  is nice

decison tree

random forest

### Modeling

Produce a predictive model that outperforms baseline in predicting the  target variable - "Finalist".

#### Baseline

In [12]:
np.mean(y_train)

TypeError: Could not convert NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoYesNoNoNoNoNoNoNoNoNoNoNoNoNoNoYesYesNoNoNoNoNoYesNoNoNoNoNoNoNoNoNoNoNoYesNoNoYesYesNoNoNoNoNoNoNoNoNoNoNoNoYesYesNoNoYesNoNoNoNoNoNoNoYesYesNoYesNoYesNoNoYesNoNoYesNoNoNoNoNoNoYesNoYesYesYesYesYesYesNoYesYesYesYesNoYesYesYesYesYesYesYesNoNoYesYesYesYesYesYesYesYesNoNoNoYesYesNoNoYesYesYesYesYesYesYesYesYesYesYesYes to numeric

In [ ]:
baseline = y_train.mean()

baseline_rmse_train =

baseline_rmse_validate = 

In [ ]:
np.median(y_train)

In [ ]:
baseline = y_train.median()

baseline_rmse_train =

baseline_rmse_validate = 

does mean or median perform better in rmse test? usewhichever performs better - the value of np.mean or np.median will be the baseline a model must beat.